In [8]:
!pip install -r requirements.txt

In [ ]:
do_train = False

In [9]:
# DO NOT MODIFY #
import torch
import random
import numpy as np
import pandas as pd
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
import sklearn
# this is how we select a GPU if it's avalible on your computer.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
from src.preprocess import clean_text 
from tqdm import tqdm
import nltk
nltk.download('punkt')
from tqdm import tqdm
from src.preprocess import clean_text

data = pd.read_pickle('our_train.pkl')
test_df = pd.read_pickle('our_test.pkl')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# to convert authors into numbers
author_to_number = {
    'EAP': 0,
    'HPL': 1,
    'MWS': 2
    
}
data["tokenized"] = data["text"].apply(lambda x: nltk.word_tokenize(clean_text(x.lower())))
test_df["tokenized"] = test_df["text"].apply(lambda x: nltk.word_tokenize(clean_text(x.lower())))

# lowercase, removing punctuation and tookenize sentences. Converting labels to int
for i in range(len(data)):
    data['author'].iloc[i] = author_to_number[data['author'].iloc[i]]
data.sample(frac=1)
for i in range(len(test_df)):
    test_df['author'].iloc[i] = author_to_number[test_df['author'].iloc[i]]
test_df.sample(frac=1)
from src.dataset import *
# Splitting dataset and generating vocab
train_df, val_df = split_train_val_test(data)
train_vocab, reversed_vocab = generate_vocab_map(train_df)
val_df.head()
test_df.head()

[nltk_data] Downloading package punkt to /home/andre/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,id,text,author,tokenized
17613,id08561,"A lamp which had been accidentally left, full ...",0,"[a, lamp, which, had, been, accidentally, left..."
17614,id01432,"I gave to each heroine of whom I read, her bea...",2,"[i, gave, to, each, heroine, of, whom, i, read..."
17615,id22037,He got in communication with Dr. Houghton of A...,1,"[he, got, in, communication, with, dr., hought..."
17616,id22330,The trees of the frequent forest belts seem to...,1,"[the, trees, of, the, frequent, forest, belts,..."
17617,id26151,"I then moved forward, and a murmuring sound ar...",2,"[i, then, moved, forward, ,, and, a, murmuring..."


In [11]:
df.head()

,id,text,author,tokenized
0,id26305,"This process, however, afforded me no means of...",0,"[this, process, ,, however, ,, afforded, me, n..."
1,id17569,It never once occurred to me that the fumbling...,1,"[it, never, once, occurred, to, me, that, the,..."
2,id11008,"In his left hand was a gold snuff box, from wh...",0,"[in, his, left, hand, was, a, gold, snuff, box..."
3,id27763,How lovely is spring As we looked from Windsor...,2,"[how, lovely, is, spring, as, we, looked, from..."
4,id12958,"Finding nothing else, not even gold, the Super...",1,"[finding, nothing, else, ,, not, even, gold, ,..."


In [12]:
df.iloc[42]

id                                                     id27080
text         It was all mud an' water, an' the sky was dark...
author                                                       1
tokenized    [it, was, all, mud, an, ', water, ,, an, ', th...
Name: 42, dtype: object

In [13]:
## TODO: complete these methods in src/dataset.py
from src.dataset import split_train_val_test, generate_vocab_map
df = df.sample(frac=1)

train_df, val_df = split_train_val_test(df)
train_vocab, reverse_vocab = generate_vocab_map(train_df)

In [14]:
from src.dataset import HeadlineDataset
from torch.utils.data import RandomSampler
#print(train_df)

train_dataset = HeadlineDataset(train_vocab, train_df)
val_dataset = HeadlineDataset(train_vocab, val_df)
test_dataset = HeadlineDataset(train_vocab, test_df)

# Now that we're wrapping our dataframes in PyTorch datsets, we can make use of PyTorch Random Samplers.
train_sampler = RandomSampler(train_dataset)
val_sampler = RandomSampler(val_dataset)
test_sampler = RandomSampler(test_dataset)

In [15]:
from torch.utils.data import DataLoader
from src.dataset import collate_fn
BATCH_SIZE = 16
train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler, collate_fn=collate_fn)
val_iterator = DataLoader(val_dataset, batch_size=BATCH_SIZE, sampler=val_sampler, collate_fn=collate_fn)
test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE, sampler=test_sampler, collate_fn=collate_fn)


In [16]:

for x, y in test_iterator:
    print(x,y)
    break
test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE, sampler=test_sampler, collate_fn=collate_fn)


tensor([[6851,  448,   82,  ...,    0,    0,    0],
        [ 101,    2, 1523,  ...,    0,    0,    0],
        [   2, 1101,    4,  ...,    0,    0,    0],
        ...,
        [ 161,    2,  247,  ...,    0,    0,    0],
        [ 174,    4, 5910,  ...,    0,    0,    0],
        [ 101,   36,    4,  ...,    0,    0,    0]]) tensor([1., 1., 2., 1., 0., 0., 2., 0., 0., 0., 0., 1., 0., 0., 0., 0.])


In [17]:
from src.models import ClassificationModel
model = None
model = ClassificationModel(len(train_vocab),embedding_dim=128,hidden_dim = 128,num_layers = 2,bidirectional = True)


In [18]:
from torch.optim import AdamW

criterion, optimizer = None, None
### YOUR CODE GOES HERE ###
criterion, optimizer = torch.nn.CrossEntropyLoss(), torch.optim.Adam(model.parameters(), lr=0.01)# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

### YOUR CODE ENDS HERE ###

In [19]:
# returns the total loss calculated from criterion
def train_loop(model, criterion, iterator):
    model.train()
    total_loss = 0
    
    for x, y in tqdm(iterator):
        optimizer.zero_grad()
        # x = x.to(device)
        # y = y.to(device)
        y = y.long()
        ### YOUR CODE STARTS HERE (~6 lines of code) ###
        prediction = model(x)
        prediction = torch.squeeze(prediction)

 
        loss = criterion(prediction,y)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    # scheduler.step()
        ### YOUR CODE ENDS HERE ###
    return total_loss

# returns:
# - true: a Python boolean array of all the ground truth values 
#         taken from the dataset iterator
# - pred: a Python boolean array of all model predictions. 
def val_loop(model, criterion, iterator):
    true, pred = [], []
    ### YOUR CODE STARTS HERE (~8 lines of code) ###
    for x, y in tqdm(iterator):
        # x = x.to(device)
        # y = y.to(device)
        # print("x",x)
        # print("y",y)  
    
        preds = model(x)
        preds = torch.squeeze(preds)
        for i_batch in range(len(y)):
            true.append(y[i_batch])
            pred.append(torch.argmax(preds[i_batch]))
            
            


    ### YOUR CODE ENDS HERE ###
    return true, pred


In [20]:
from sklearn.metrics import f1_score, accuracy_score
# To test your eval implementation, let's see how well the untrained model does on our dev dataset.
# It should do pretty poorly.
from src.eval_utils import binary_macro_f1, accuracy
true, pred = val_loop(model, criterion, val_iterator)
true = [x.item() for x in true]
pred = [x.item() for x in pred]

print(f1_score(true, pred, average='weighted'))
print(accuracy_score(true, pred))


100%|██████████| 123/123 [00:06<00:00, 19.16it/s]

0.2915791559395867
0.3263534218590398


In [21]:
if do_train:
    TOTAL_EPOCHS = 7
    for epoch in range(TOTAL_EPOCHS):
        train_loss = train_loop(model, criterion, train_iterator)
        true, pred = val_loop(model, criterion, val_iterator)
        print(f"EPOCH: {epoch}")
        print(f"TRAIN LOSS: {train_loss}")
        print(f"VAL F-1: {f1_score(true, pred, average='weighted')}")
        print(f"VAL ACC: {accuracy_score(true, pred)}")
    file = open('custom_embeddings.model', 'w+')    
    torch.save(model.state_dict(), f'custom_embeddings.model')


100%|██████████| 123/123 [00:07<00:00, 16.37it/s]


EPOCH: 0
TRAIN LOSS: 880.5618799179792
VAL F-1: 0.721644255599444
VAL ACC: 0.7236976506639428


100%|██████████| 123/123 [00:07<00:00, 16.03it/s]


EPOCH: 1
TRAIN LOSS: 562.099752612412
VAL F-1: 0.7553912422501342
VAL ACC: 0.7553626149131767


100%|██████████| 123/123 [00:06<00:00, 18.58it/s]


...

EPOCH: 4
TRAIN LOSS: 410.00900747440755
VAL F-1: 0.7420459583746026
VAL ACC: 0.7415730337078652


100%|██████████| 123/123 [00:06<00:00, 18.00it/s]


EPOCH: 5
TRAIN LOSS: 378.71402362920344
VAL F-1: 0.7384054405505677
VAL ACC: 0.7390194075587334


100%|██████████| 123/123 [00:07<00:00, 17.52it/s]


EPOCH: 6
TRAIN LOSS: 412.0156566388905
VAL F-1: 0.7348490368971985
VAL ACC: 0.7354443309499489


In [22]:
model.load_state_dict(torch.load('custom_embeddings.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [23]:
true, pred = val_loop(model, criterion, test_iterator)
print(f"TEST F-1: {f1_score(true, pred, average='weighted')}")
print(f"TEST ACC: {accuracy_score(true, pred)}")

100%|██████████| 123/123 [00:06<00:00, 18.02it/s]

TEST F-1: 0.8768857919934083
TEST ACC: 0.8769074262461851
